# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

spark = SparkSession.builder.appName('lin_reg_project').getOrCreate()

In [0]:
data = spark.read.csv('/FileStore/tables/cruise_ship_info.csv',header=True,inferSchema=True)
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [0]:
data.columns

Out[7]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [0]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [0]:
#Trying out predictions without Cruise Line feature
assembler = VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density'],outputCol='features')
new_data = assembler.transform(data)
final_data = new_data.select('features','crew')
train_data, test_data = final_data.randomSplit([0.7,0.3])


In [0]:
#Creating the model
lr = LinearRegression(featuresCol='features',labelCol='crew',predictionCol='predictions')
lr_model = lr.fit(train_data)
test_results = lr_model.evaluate(test_data)

In [0]:
test_results.r2

Out[10]: 0.9611007147638599

In [0]:
test_results.rootMeanSquaredError

Out[11]: 0.6910940341081985

In [0]:
#Including the cruise line using String Indexer
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [0]:
cruise_indexer = StringIndexer(inputCol='Cruise_line',outputCol='Cruise_line_indexed')
data_new = cruise_indexer.fit(data).transform(data)
data_new.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_indexed|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|                1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|                1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|                1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.5

In [0]:
#Trying out predictions with Cruise Line feature
assembler = VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density','Cruise_line_indexed'],outputCol='features')
new_data = assembler.transform(data_new)
final_data = new_data.select('features','crew')
train_data, test_data = final_data.randomSplit([0.7,0.3])

lr = LinearRegression(featuresCol='features',labelCol='crew',predictionCol='predictions')
lr_model = lr.fit(train_data)
test_results = lr_model.evaluate(test_data)

In [0]:
test_results.r2

Out[20]: 0.9095331373068362

In [0]:
test_results.rootMeanSquaredError

Out[21]: 0.9506061253548305

In [0]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               126|
|   mean| 7.780634920634934|
| stddev|3.5858936394782814|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [0]:
#Checking correlations between the features and crew to verify our results:
from pyspark.sql.functions import corr
features = ['Cruise_line_indexed','Age','Tonnage','passengers','length','cabins','passenger_density']
for feature in features:
    data_new.select(corr(feature,'crew')).show()

+-------------------------------+
|corr(Cruise_line_indexed, crew)|
+-------------------------------+
|            -0.5154627327113318|
+-------------------------------+

+-------------------+
|    corr(Age, crew)|
+-------------------+
|-0.5306565039638852|
+-------------------+

+-------------------+
|corr(Tonnage, crew)|
+-------------------+
| 0.9275688115449388|
+-------------------+

+----------------------+
|corr(passengers, crew)|
+----------------------+
|    0.9152341306065384|
+----------------------+

+------------------+
|corr(length, crew)|
+------------------+
| 0.895856627101658|
+------------------+

+------------------+
|corr(cabins, crew)|
+------------------+
|0.9508226063578497|
+------------------+

+-----------------------------+
|corr(passenger_density, crew)|
+-----------------------------+
|         -0.15550928421699717|
+-----------------------------+

